In [1]:
import math
import random
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.distributions
from collections import namedtuple
from itertools import count

device = "cuda:1"

batchsize = 512
nsamples = 8
npoints = 5
emsize = 512


class Graph_Transformer(nn.Module):
    def __init__(self, emsize = 64, nhead = 8, nhid = 1024, nlayers = 4, ndecoderlayers = 2, dropout = 0):
        super().__init__()
        self.emsize = emsize
        from torch.nn import TransformerEncoder, TransformerEncoderLayer, TransformerDecoder, TransformerDecoderLayer
        encoder_layers = TransformerEncoderLayer(emsize, nhead, nhid, dropout = dropout)
        decoder_layers = TransformerDecoderLayer(emsize, nhead, nhid, dropout = dropout)
        self.transformer_encoder = TransformerEncoder(encoder_layers, nlayers)
        self.transformer_decoder = TransformerDecoder(decoder_layers, ndecoderlayers)
        self.encoder = nn.Linear(2, emsize)
        self.outputattention_query = nn.Linear(emsize, emsize, bias = False)
        self.outputattention_key = nn.Linear(emsize, emsize, bias = False)
        self.start_token = nn.Parameter(torch.randn([emsize], device = device))
        
        self.lineartest0 = nn.Linear(2, emsize)
        self.lineartest1 = nn.Linear(2, emsize)
        self.lineartest2 = nn.Linear(2, emsize)
        self.lineartest3 = nn.Linear(2 * emsize, emsize)
    
    def generate_subsequent_mask(self, sz): #last dimension will be softmaxed over when adding to attention logits, if boolean the ones turn into -inf
        #mask = (torch.triu(torch.ones(sz, sz)) == 1).transpose(0, 1)
        #mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))
        
        mask = torch.triu(torch.ones([sz, sz], dtype = torch.bool, device = device), diagonal = 1)
        return mask
    
    def encode(self, src): #src must be [batchsize * nsamples, npoints, 2]
        src = self.encoder(src).transpose(0, 1)
        output = self.transformer_encoder(src)
        return output #[npoints, batchsize * nsamples, emsize]
    
    def test_encode(self, src): #src must be [batchsize * nsamples, npoints = 5, 2]
        point_1 = self.lineartest1(src[:, 3, :] - 0.5) #want mean 0!!!!
        point_2 = self.lineartest1(src[:, 4, :] - 0.5)
        remaining = self.lineartest0(src[:, :3, :]) #[batchsize * nsamples, 3, emsize]
        point_1_message = self.lineartest1(src[:, 3, :] - 0.5).squeeze(1)
        point_2_message = self.lineartest2(src[:, 4, :] - 0.5).squeeze(1)
        point_1 = F.relu(torch.cat([point_1, point_2_message], dim = 1))
        point_2 = F.relu(torch.cat([point_2, point_1_message], dim = 1))
        point_1 = self.lineartest3(point_1)
        point_2 = self.lineartest3(point_2)
        src = torch.cat([torch.zeros_like(remaining.transpose(0, 1)), point_1.unsqueeze(0), point_2.unsqueeze(0)])
        return src #[npoints = 5, batchsize * nsamples, emsize]
    
    def decode_next(self, memory, tgt, route_mask): #route mask is [batchsize * nsamples, npoints], both memory and tgt must have batchsize and nsamples in same dimension (the 1th one)
        npoints = memory.size(0)
        batchsize = tgt.size(1)
        """if I really wanted this to be efficient I'd only recompute the decoder for the last tgt, and just remebering what the others looked like from before (won't change due to mask)"""
        """have the option to freeze the autograd on all but the last part of tgt, although at the moment this is a very natural way to say: initial choices matter more"""
        tgt_mask = self.generate_subsequent_mask(tgt.size(0))
        output = self.transformer_decoder(tgt, memory, tgt_mask) #[tgt, batchsize * nsamples, emsize]
        output_query = self.outputattention_query(memory).transpose(0, 1) #[batchsize * nsamples, npoints, emsize]
        output_key = self.outputattention_key(output[-1]) #[batchsize * nsamples, emsize]
        output_attention = torch.matmul(output_query * self.emsize ** -0.5, output_key.unsqueeze(-1)).squeeze(-1) #[batchsize * nsamples, npoints], technically don't need to scale attention as we divide by variance next anyway
        output_attention_tanh = output_attention.tanh() #[batchsize * nsamples, npoints]
        
        #we clone the route_mask incase we want to backprop using it (else it was modified by inplace opporations)
        output_attention = output_attention.masked_fill(route_mask.clone(), float('-inf')) #[batchsize * nsamples, npoints]
        output_attention_tanh = output_attention_tanh.masked_fill(route_mask.clone(), float('-inf')) #[batchsize * nsamples, npoints]
        
        return output_attention_tanh, output_attention #[batchsize * nsamples, npoints]
    
    def calculate_logprob(self, memory, routes): #memory is [npoints, batchsize * nsamples, emsize], routes is [batchsize * nsamples, npoints - 3], rather than backproping the entire loop, this saves vram (and computation)
        npoints = memory.size(0)
        ninternalpoints = routes.size(1)
        bigbatchsize = memory.size(1)
        memory_ = memory.gather(0, routes.transpose(0, 1).unsqueeze(2).expand(-1, -1, self.emsize)) #[npoints - 3, batchsize * nsamples, emsize] reorder memory into order of routes
        tgt = torch.cat([self.start_token.unsqueeze(0).unsqueeze(1).expand(1, bigbatchsize, -1), memory_[:-1]]) #[npoints - 3, batchsize * nroutes, emsize], want to go from memory to tgt
        tgt_mask = self.generate_subsequent_mask(ninternalpoints)
        output = self.transformer_decoder(tgt, memory, tgt_mask) #[npoints - 3, batchsize * nsamples, emsize]
        """want probability of going from key to query, but first need to normalise (softmax with mask)"""
        output_query = self.outputattention_query(memory_).transpose(0, 1) #[batchsize * nsamples, npoints - 3, emsize]
        output_key = self.outputattention_key(output).transpose(0, 1) #[batchsize * nsamples, npoints - 3, emsize]
        attention_mask = torch.full([ninternalpoints, ninternalpoints], True, dtype = torch.bool, device = device).triu(1) #[npoints - 3, npoints - 3], True for i < j
        output_attention = torch.matmul(output_query * self.emsize ** -0.5, output_key.transpose(-1, -2))
        """quick fix to stop divergence"""
        output_attention_tanh = output_attention.tanh()
        
        output_attention_tanh = output_attention_tanh.masked_fill(attention_mask, float('-inf'))
        output_attention_tanh = output_attention_tanh - output_attention_tanh.logsumexp(-2, keepdim = True) #[batchsize * nsamples, npoints - 3, npoints - 3]
        
        output_attention = output_attention.masked_fill(attention_mask, float('-inf'))
        output_attention = output_attention - output_attention.logsumexp(-2, keepdim = True) #[batchsize * nsamples, npoints - 3, npoints - 3]
        
        """infact I'm almost tempted to not mask choosing a previous point, so it's forced to learn it and somehow incorporate it into its computation, but without much impact on reinforcing good examples"""
        logprob_tanh = output_attention_tanh.diagonal(dim1 = -1, dim2 = -2).sum(-1) #[batchsize * nsamples]
        logprob = output_attention.diagonal(dim1 = -1, dim2 = -2).sum(-1) #[batchsize * nsamples]
        return logprob_tanh, logprob #[batchsize * nsamples]

NN = Graph_Transformer().to(device)
optimizer = optim.Adam(NN.parameters())


class environment():
    def reset(self, npoints, batchsize, nsamples):
        if npoints <= 3:
            print("Error: not enough points for valid problem instance")
            return
        self.batchsize = batchsize * nsamples #so that I don't have to rewrite all this code, we store these two dimensions together
        self.nsamples = nsamples
        self.npoints = npoints
        self.points = torch.rand([batchsize, npoints - 3, 2], device = device).unsqueeze(1).expand(-1, nsamples, -1, -1).reshape(self.batchsize, npoints - 3, 2)
        self.corner_points = torch.tensor([[0, 0], [2, 0], [0, 2]], dtype = torch.float, device = device)
        self.points = torch.cat([self.corner_points.unsqueeze(0).expand(self.batchsize, -1, -1), self.points], dim = -2) #[batchsize * nsamples, npoints, 2]
        self.points_mask = torch.cat([torch.ones([self.batchsize, 3], dtype = torch.bool, device = device), torch.zeros([self.batchsize, npoints - 3], dtype = torch.bool, device = device)], dim = 1)
        self.points_sequence = torch.empty([self.batchsize, 0], dtype = torch.long, device = device)
        
        """use a trick, for the purpose of an 'external' triangle that is always left untouched, which means we don't have to deal with boundary edges as being different. external triangle is [0, 1, 2] traversed clockwise..."""
        self.partial_delaunay_triangles = torch.tensor([[0, 2, 1], [0, 1, 2]], dtype = torch.int64, device = device).unsqueeze(0).expand(self.batchsize, -1, -1).contiguous() #[batchsize, ntriangles, 3] contains index of points, always anticlockwise
        self.partial_delaunay_edges = torch.tensor([5, 4, 3, 2, 1, 0], dtype = torch.int64, device = device).unsqueeze(0).expand(self.batchsize, -1).contiguous() #[batchsize, ntriangles * 3] contains location of corresponding edge (edges go in order 01, 12, 20). Edges will always flip since triangles are stored anticlockwise.
        
        self.ntriangles = 2 #can store as scalar, since will always be the same
        self.cost = torch.zeros([self.batchsize], device = device)
        
        self.logprob = torch.zeros([self.batchsize], device = device, requires_grad = True)
    
    def update(self, point_index): #point_index is [batchsize]
        if point_index.size(0) != self.batchsize:
            print("Error: point_index.size() doesn't match expected size, should be [batchsize]")
            return
        if self.points_mask.gather(1, point_index.unsqueeze(1)).sum():
            print("Error: some points already added")
            return
        triangles_coordinates = self.points.gather(1, self.partial_delaunay_triangles.view(self.batchsize, self.ntriangles * 3).unsqueeze(2).expand(-1, -1, 2)).view(self.batchsize, self.ntriangles, 3, 2) #[batchsize, ntriangles, 3, 2]
        newpoint = self.points.gather(1, point_index.unsqueeze(1).unsqueeze(2).expand(self.batchsize, 1, 2)).squeeze(1) #[batchsize, 2]
        
        incircle_matrix = torch.cat([triangles_coordinates, newpoint.unsqueeze(1).unsqueeze(2).expand(-1, self.ntriangles, 1, -1)], dim = -2) #[batchsize, ntriangles, 4, 2]
        incircle_matrix = torch.cat([incircle_matrix, (incircle_matrix * incircle_matrix).sum(-1, keepdim = True), torch.ones([self.batchsize, self.ntriangles, 4, 1], device = device)], dim = -1) #[batchsize, ntriangles, 4, 4]
        incircle_test = incircle_matrix.det() > 0 #[batchsize, ntriangles], is True if inside incircle
        removed_edge_mask = incircle_test.unsqueeze(2).expand(-1, -1, 3).reshape(-1) #[batchsize * ntriangles * 3]
        
        edges = (self.partial_delaunay_edges + self.ntriangles * 3 * torch.arange(self.batchsize, device = device).unsqueeze(1)).view(-1) #[batchsize * ntriangles * 3]
        neighbouring_edge = edges.masked_select(removed_edge_mask)
        neighbouring_edge_mask = torch.zeros([self.batchsize * self.ntriangles * 3], device = device, dtype = torch.bool)
        neighbouring_edge_mask[neighbouring_edge] = True
        neighbouring_edge_mask = (neighbouring_edge_mask * removed_edge_mask.logical_not()) #[batchsize * ntriangles * 3]
        
        n_new_triangles = neighbouring_edge_mask.view(self.batchsize, -1).sum(-1) #[batchsize]
        
        new_point = point_index.unsqueeze(1).expand(-1, self.ntriangles * 3).masked_select(neighbouring_edge_mask.view(self.batchsize, -1))
        
        second_point_mask = neighbouring_edge_mask.view(self.batchsize, -1, 3) #[batchsize, ntriangles 3]
        (first_point_indices0, first_point_indices1, first_point_indices2) = second_point_mask.nonzero(as_tuple = True)
        first_point_indices2 = (first_point_indices2 != 2) * (first_point_indices2 + 1)
        
        first_point = self.partial_delaunay_triangles[first_point_indices0, first_point_indices1, first_point_indices2] #[?]
        second_point = self.partial_delaunay_triangles.masked_select(second_point_mask) #[?]
        
        new_triangles_mask = torch.cat([incircle_test, torch.ones([self.batchsize, 2], dtype = torch.bool, device = device)], dim = 1) #[batchsize, ntriangles + 2]
        
        new_neighbouring_edges = 3 * new_triangles_mask.nonzero(as_tuple = True)[1] #[?], 3* since is the 01 edge of new triangles (see later)
        self.partial_delaunay_edges.masked_scatter_(neighbouring_edge_mask.view(self.batchsize, -1), new_neighbouring_edges) #still [batchsize, ntriangles * 3] for now
        
        self.partial_delaunay_triangles = torch.cat([self.partial_delaunay_triangles, torch.empty([self.batchsize, 2, 3], dtype = torch.long, device = device)], dim = 1)
        self.partial_delaunay_edges = torch.cat([self.partial_delaunay_edges, torch.empty([self.batchsize, 6], dtype = torch.long, device = device)], dim = 1)
        new_triangles = torch.stack([first_point, second_point, new_point], dim = 1) #[?, 3], edge here is flipped compared to edge in neighbouring triangle (so first_point is the second point in neighbouring edge)
        self.partial_delaunay_triangles.masked_scatter_(new_triangles_mask.unsqueeze(2).expand(-1, -1, 3), new_triangles) #[batchsize, ntriangles + 2, 3]
        
        new_edge01 = neighbouring_edge_mask.view(self.batchsize, -1).nonzero(as_tuple = True)[1] #[?]
        
        """we are currently storing which triangles have to be inserted, via the edges along the perimeter of the delaunay cavity, we need to compute which edge is to the 'left'/'right' of each edge"""
        """don't have the memory to do a batchsize * n * n boolean search, don't have the speed to do a batchsize^2 search (as would be the case for sparse matrix or similar)"""
        """best alternative: rotate the edge around right point, repeat until hit edge in mask (will never go to an edge of a removed triangle before we hit edge in mask) should basically be order 1!!!!!"""
        
        neighbouring_edge_index = neighbouring_edge_mask.nonzero(as_tuple = True)[0] #[?]
        next_neighbouring_edge_index = torch.empty_like(neighbouring_edge_index) #[?]
        
        rotating_flipped_neighbouring_edge_index = neighbouring_edge_mask.nonzero(as_tuple = True)[0] #[?], initialise
        todo_mask = torch.ones_like(next_neighbouring_edge_index, dtype = torch.bool) #[?]
        while todo_mask.sum():
            rotating_neighbouring_edge_index = rotating_flipped_neighbouring_edge_index + 1 - 3 * (rotating_flipped_neighbouring_edge_index % 3 == 2) #[todo_mask.sum()], gets smaller until nothing left EFFICIENCY (this may be seriously stupid, as it requires making a bunch of copies when I could be doing stuff inplace)
            
            update_mask = neighbouring_edge_mask[rotating_neighbouring_edge_index] #[todo_mask.sum()]
            update_mask_unravel = torch.zeros_like(todo_mask).masked_scatter(todo_mask, update_mask) #[?]
            
            next_neighbouring_edge_index.masked_scatter_(update_mask_unravel, rotating_neighbouring_edge_index.masked_select(update_mask)) #[?]
            
            todo_mask.masked_fill_(update_mask_unravel, False) #[?]
            rotating_flipped_neighbouring_edge_index = edges[rotating_neighbouring_edge_index.masked_select(update_mask.logical_not())] #[todo_mask.sum()]
        triangle_index = new_triangles_mask.view(-1).nonzero(as_tuple = True)[0] #[?], index goes up to batchsize * (ntriangles + 2), this is needed for when we invert the permutation by scattering (won't scatter same number of triangles per batch)
        
        next_triangle_index = torch.empty_like(edges).masked_scatter_(neighbouring_edge_mask, triangle_index)[next_neighbouring_edge_index] #[?], index goes up to batchsize * (ntriangles + 2)
        next_edge = 3 * next_triangle_index + 1 #[?]
        
        invert_permutation = torch.empty_like(new_triangles_mask.view(-1), dtype=torch.long) #[batchsize * (ntriangles + 2)]
        invert_permutation[next_triangle_index] = triangle_index #[batchsize * (ntriangles + 2)]
        previous_triangle_index = invert_permutation.masked_select(new_triangles_mask.view(-1)) #[?]
        previous_edge = 3 * previous_triangle_index + 2 #[?]
        
        """in the above we rotated around 'first_point' in our new triangles"""
        new_edge20 = next_edge % ((self.ntriangles + 2) * 3) #[?]
        new_edge12 = previous_edge % ((self.ntriangles + 2) * 3) #[?]
        
        new_edges = torch.stack([new_edge01, new_edge12, new_edge20], dim = 1) #[?, 3]
        self.partial_delaunay_edges.masked_scatter_(new_triangles_mask.unsqueeze(2).expand(-1, -1, 3).reshape(self.batchsize, -1), new_edges) #[batchsize, (ntriangles + 2) * 3]
        
        self.ntriangles += 2
        """currently only count the extra triangles you replace (not the on you have to remove because you're located there, and not the ones you make because you have to create two more"""
        self.cost += (n_new_triangles - 3)
        self.points_mask.scatter_(1, point_index.unsqueeze(1).expand(-1, self.npoints), True)
        self.points_sequence = torch.cat([self.points_sequence, point_index.unsqueeze(1)], dim = 1)
    
    def sample_point(self, logits): #logits must be [batchsize * nsamples, npoints]
        probs = torch.distributions.categorical.Categorical(logits = logits)
        next_point = probs.sample() #size is [batchsize * nsamples]
        self.update(next_point)
        self.logprob = self.logprob + probs.log_prob(next_point)
        return next_point #[batchsize * nsamples]
    
    def sampleandgreedy_point(self, logits): #logits must be [batchsize * nsamples, npoints], last sample will be the greedy choice (but we still need to keep track of its logits)
        logits_sample = logits.view(-1, self.nsamples, self.npoints)[:, :-1, :]
        probs = torch.distributions.categorical.Categorical(logits = logits_sample)
        
        sample_point = probs.sample() #[batchsize, (nsamples - 1)]
        greedy_point = logits.view(-1, self.nsamples, self.npoints)[:, -1, :].max(-1, keepdim = True)[1] #[batchsize, 1]
        next_point = torch.cat([sample_point, greedy_point], dim = 1).view(-1)
        self.update(next_point)
        self.logprob = self.logprob + torch.cat([probs.log_prob(sample_point), torch.zeros([sample_point.size(0), 1], device = device)], dim = 1).view(-1)
        return next_point
    

env = environment()


def train(epochs = 300000, npoints = 13, batchsize = 2000, nsamples = 8):
    NN.train()
    for i in range(epochs):
        env.reset(npoints, batchsize, nsamples)
        """include the boundary points, kinda makes sense that they should contribute (atm only in the encoder, difficult to see how in the decoder)"""
        memory = NN.encode(env.points) #[npoints, batchsize * nsamples, emsize]
        #### #### #### remember to include tgt.detach() when reinstate with torch.no_grad()
        tgt = NN.start_token.unsqueeze(0).unsqueeze(1).expand(1, batchsize * nsamples, -1).detach() #[1, batchsize * nsamples, emsize]
        #with torch.no_grad(): #to speed up computation, selecting routes is done without gradient
        NN.eval()
        with torch.no_grad():
            for j in range(3, npoints):
                #### #### #### remember to include memory.detach() when reinstate with torch.no_grad()
                _, logits = NN.decode_next(memory.detach(), tgt, env.points_mask)
                next_point = env.sampleandgreedy_point(logits)
                """
                for inputing the previous embedding into decoder
                """
                tgt = torch.cat([tgt, memory.gather(0, next_point.unsqueeze(0).unsqueeze(2).expand(1, -1, memory.size(2)))]) #[nsofar, batchsize * nsamples, emsize]
                """
                for inputing the previous decoder output into the decoder (allows for an evolving strategy, but doesn't allow for fast training
                """
                ############

        
        _, logprob = NN.calculate_logprob(memory, env.points_sequence) #[batchsize * nsamples]
        NN.train()
        """
        clip logprob so doesn't reinforce things it already knows
        TBH WANT SOMETHING DIFFERENT ... want to massively increase training if find something unexpected and otherwise not
        """
        greedy_prob = logprob.view(batchsize, nsamples)[:, -1].detach() #[batchsize]
        greedy_baseline = env.cost.view(batchsize, nsamples)[:, -1] #[batchsize], greedy sample
        fixed_baseline = 0.5 * torch.ones([1], device = device)
        min_baseline = env.cost.view(batchsize, nsamples)[:, :-1].min(-1)[0] #[batchsize], minimum cost
        baseline = greedy_baseline
        regular_reinforcement = env.cost.view(batchsize, nsamples)[:, :-1] - baseline.unsqueeze(1)
        positive_reinforcement = - F.relu( - (env.cost.view(batchsize, nsamples)[:, :-1] - baseline.unsqueeze(1))) #don't scale positive reinforcement
        negative_reinforcement = F.relu(env.cost.view(batchsize, nsamples)[:, :-1] - baseline.unsqueeze(1))
        positive_reinforcement_binary = env.cost.view(batchsize, nsamples)[:, :-1] - baseline.unsqueeze(1) <= -0.05
        negative_reinforcement_binary = env.cost.view(batchsize, nsamples)[:, :-1] - baseline.unsqueeze(1) > 2
        """
        binary positive reinforcement
        """
        #loss = - ((logprob.view(batchsize, nsamples)[:, :-1] < -0.2) * logprob.view(batchsize, nsamples)[:, :-1] * positive_reinforcement_binary).mean() #+ (logprob.view(batchsize, nsamples)[:, :-1] > -1) * logprob.view(batchsize, nsamples)[:, :-1] * negative_reinforcement_binary
        """
        clipped binary reinforcement
        """
        #loss = ( 
        #        - logprob.view(batchsize, nsamples)[:, :-1] 
        #        #* (logprob.view(batchsize, nsamples)[:, :-1] < 0) 
        #        * positive_reinforcement_binary 
        #        + logprob.view(batchsize, nsamples)[:, :-1] 
        #        #* (logprob.view(batchsize, nsamples)[:, :-1] > greedy_prob.unsqueeze(1) - 25) 
        #        * negative_reinforcement_binary 
        #).mean()
        """
        clipped binary postive, clipped weighted negative
        """
        #loss = ( - logprob.view(batchsize, nsamples)[:, :-1] * (logprob.view(batchsize, nsamples)[:, :-1] < -0.2) * positive_reinforcement_binary + logprob.view(batchsize, nsamples)[:, :-1] * (logprob.view(batchsize, nsamples)[:, :-1] > -2) * negative_reinforcement ).mean()
        """
        clipped reinforcement without rescaling
        """
        #loss = ((logprob.view(batchsize, nsamples)[:, :-1] < -0.7) * logprob.view(batchsize, nsamples)[:, :-1] * positive_reinforcement + (logprob.view(batchsize, nsamples)[:, :-1] > -5) * logprob.view(batchsize, nsamples)[:, :-1] * negative_reinforcement).mean()
        """
        clipped reinforcement
        """
        #loss = (logprob.view(batchsize, nsamples)[:, :-1] * positive_reinforcement / (positive_reinforcement.var() + 0.001).sqrt() + (logprob.view(batchsize, nsamples)[:, :-1] > -3) * logprob.view(batchsize, nsamples)[:, :-1] * negative_reinforcement / (negative_reinforcement.var() + 0.001).sqrt()).mean()
        """
        balanced reinforcement
        """
        #loss = (logprob.view(batchsize, nsamples)[:, :-1] * (positive_reinforcement / (positive_reinforcement.var() + 0.001).sqrt() + negative_reinforcement / (negative_reinforcement.var() + 0.001).sqrt())).mean()
        """
        regular loss with thresholding
        """
        loss = (logprob.view(batchsize, nsamples)[:, :-1] * regular_reinforcement * (positive_reinforcement_binary + negative_reinforcement_binary * (logprob.view(batchsize, nsamples)[:, :-1] > greedy_prob.unsqueeze(1) - 40))).mean()
        """
        regular loss
        """
        #loss = (logprob.view(batchsize, nsamples)[:, :-1] * (positive_reinforcement + negative_reinforcement)).mean()
        optimizer.zero_grad()
        loss.backward()
        #print(NN.encoder.weight.grad)
        optimizer.step()
        #print(greedy_baseline.mean().item())
        print(greedy_baseline.mean().item(), logprob.view(batchsize, nsamples)[:, -1].mean().item(), logprob.view(batchsize, nsamples)[:, :-1].mean().item(), logprob[batchsize - 1].item(), logprob[0].item())#, env.logprob[0].item())
  

In [ ]:
train(npoints = 103, batchsize = 150, nsamples = 8)

281.96002197265625 -361.68145751953125 -363.7059020996094 -364.3961181640625 -363.4555969238281
325.7933349609375 -347.0340270996094 -361.7479553222656 -364.5716552734375 -364.341796875
296.5 -354.3435363769531 -363.1494140625 -361.9560546875 -363.9110412597656
272.3600158691406 -358.7971496582031 -363.5966491699219 -362.78350830078125 -363.69921875
297.9866638183594 -360.8554992675781 -363.6600036621094 -363.87445068359375 -363.7345886230469
294.64666748046875 -357.14251708984375 -363.4405517578125 -364.34063720703125 -362.9004211425781
288.3666687011719 -353.3299560546875 -362.9281311035156 -362.5955505371094 -362.38189697265625
290.0 -343.0948791503906 -360.6539001464844 -361.48468017578125 -358.6470947265625
290.1199951171875 -316.81573486328125 -349.2180480957031 -346.94287109375 -349.41595458984375
271.67333984375 -305.3627014160156 -341.7922668457031 -343.05303955078125 -345.86126708984375
274.8600158691406 -268.1111145019531 -312.5824279785156 -308.9215393066406 -300.0108032226

293.0400085449219 -363.539794921875 -363.7384338378906 -363.7442626953125 -363.745849609375
292.6933288574219 -363.53619384765625 -363.7382507324219 -363.7236328125 -363.7235107421875
292.17333984375 -363.52752685546875 -363.73828125 -363.757568359375 -363.74505615234375
294.0533447265625 -363.518798828125 -363.73992919921875 -363.752685546875 -363.75299072265625
293.32666015625 -363.5140380859375 -363.7405700683594 -363.7967529296875 -363.75372314453125
294.23333740234375 -363.51446533203125 -363.7392272949219 -363.7562255859375 -363.7330322265625
293.1533508300781 -363.5115661621094 -363.73858642578125 -363.7476806640625 -363.75640869140625
292.58001708984375 -363.50653076171875 -363.7389831542969 -363.7329406738281 -363.7396240234375
292.20001220703125 -363.5066223144531 -363.73919677734375 -363.75079345703125 -363.686279296875
291.14666748046875 -363.5048828125 -363.7396545410156 -363.70513916015625 -363.751953125
292.7733459472656 -363.5054016113281 -363.7396545410156 -363.7276000

282.17999267578125 -363.4909362792969 -363.739013671875 -363.7177429199219 -363.72613525390625
281.3333435058594 -363.4901123046875 -363.7394104003906 -363.73712158203125 -363.7171325683594
280.8333435058594 -363.4930419921875 -363.73834228515625 -363.72747802734375 -363.73187255859375
279.933349609375 -363.4927062988281 -363.7392272949219 -363.71673583984375 -363.72601318359375
280.4800109863281 -363.48681640625 -363.7391357421875 -363.70013427734375 -363.767333984375
279.8866882324219 -363.4832763671875 -363.738037109375 -363.7596740722656 -363.7044982910156
279.9266662597656 -363.4764404296875 -363.7388916015625 -363.7203063964844 -363.68280029296875
278.76666259765625 -363.4795837402344 -363.73779296875 -363.7414245605469 -363.72576904296875
277.32666015625 -363.47344970703125 -363.7381591796875 -363.763916015625 -363.7558288574219
276.0733337402344 -363.4662170410156 -363.73968505859375 -363.7138671875 -363.75457763671875
275.3999938964844 -363.458251953125 -363.73822021484375 -36

259.7799987792969 -340.1630554199219 -359.660400390625 -358.1671447753906 -359.69512939453125
260.70001220703125 -344.833984375 -361.21282958984375 -360.03369140625 -360.6259765625
262.02667236328125 -348.7849426269531 -362.1912841796875 -364.40582275390625 -363.98406982421875
264.5066833496094 -351.3240966796875 -362.62933349609375 -362.2986755371094 -362.718994140625
264.9666748046875 -351.69476318359375 -362.8024597167969 -364.751953125 -361.69580078125
265.44000244140625 -350.88165283203125 -362.62432861328125 -362.51861572265625 -364.7872314453125
266.2066650390625 -346.1548156738281 -361.4296875 -362.290771484375 -361.7987060546875
263.7533264160156 -334.9143371582031 -356.81243896484375 -353.2304382324219 -356.0938720703125
260.9599914550781 -322.1556701660156 -349.3731384277344 -349.31402587890625 -354.920654296875
257.3999938964844 -292.5299987792969 -327.9305114746094 -325.9149169921875 -318.89215087890625
258.87335205078125 -259.5795593261719 -302.47601318359375 -324.7555541

247.9666748046875 -360.84521484375 -363.6999816894531 -363.6123046875 -363.47955322265625
248.5066680908203 -360.48553466796875 -363.692626953125 -363.9896545410156 -363.7056884765625
246.41334533691406 -359.85943603515625 -363.66204833984375 -364.01043701171875 -364.1153564453125
247.67333984375 -358.5403747558594 -363.5967712402344 -362.9169006347656 -363.49639892578125
248.48667907714844 -357.077392578125 -363.4559631347656 -363.16522216796875 -362.892822265625
244.36666870117188 -354.8921813964844 -363.2837219238281 -364.552978515625 -363.8858642578125
246.68667602539062 -351.17156982421875 -362.79058837890625 -360.4154052734375 -363.05413818359375
246.5066680908203 -341.0486145019531 -360.1792297363281 -360.40020751953125 -362.4399108886719
277.32666015625 -348.23291015625 -360.57708740234375 -360.05718994140625 -356.7262268066406
352.2133483886719 -344.64666748046875 -360.51617431640625 -361.13409423828125 -357.635009765625
351.8800048828125 -363.00262451171875 -363.7372131347656

...

321.40667724609375 -363.727783203125 -363.73931884765625 -363.7399597167969 -363.74310302734375
320.0933532714844 -363.7278137207031 -363.7393493652344 -363.7408447265625 -363.739990234375
321.7533264160156 -363.727783203125 -363.7393493652344 -363.73974609375 -363.7376708984375
320.5933532714844 -363.7278137207031 -363.7393798828125 -363.738037109375 -363.7384033203125
320.2200012207031 -363.72784423828125 -363.73931884765625 -363.74212646484375 -363.7381286621094
316.23333740234375 -363.7278137207031 -363.7393798828125 -363.738525390625 -363.7392578125
317.70001220703125 -363.727783203125 -363.7393493652344 -363.7381591796875 -363.7376403808594
320.20001220703125 -363.72772216796875 -363.73931884765625 -363.73883056640625 -363.739990234375
317.9866638183594 -363.727783203125 -363.7393493652344 -363.7381591796875 -363.73870849609375
321.37335205078125 -363.72772216796875 -363.7393493652344 -363.73944091796875 -363.73944091796875
320.5733337402344 -363.72772216796875 -363.7393493652344

318.2533264160156 -363.7285461425781 -363.7393493652344 -363.738037109375 -363.7390441894531
319.0133361816406 -363.7285461425781 -363.7394104003906 -363.739990234375 -363.74017333984375
318.3333435058594 -363.7285461425781 -363.73931884765625 -363.7413330078125 -363.73846435546875
320.02001953125 -363.7285461425781 -363.7393798828125 -363.74029541015625 -363.7391357421875
318.55999755859375 -363.7285461425781 -363.7394104003906 -363.7413635253906 -363.7403869628906
318.08001708984375 -363.7284851074219 -363.7393493652344 -363.73748779296875 -363.7386474609375
318.82000732421875 -363.72845458984375 -363.7394104003906 -363.7383117675781 -363.73846435546875
320.3000183105469 -363.7284851074219 -363.7394104003906 -363.73974609375 -363.7405090332031
318.5333557128906 -363.7285461425781 -363.7393798828125 -363.73779296875 -363.73956298828125
317.94000244140625 -363.72845458984375 -363.7392883300781 -363.7412109375 -363.7396240234375
317.5733337402344 -363.72845458984375 -363.7394104003906 -

319.02667236328125 -363.7286682128906 -363.7392883300781 -363.73663330078125 -363.739501953125
317.3333435058594 -363.7286071777344 -363.7393493652344 -363.73779296875 -363.740966796875
318.6199951171875 -363.7286071777344 -363.7394104003906 -363.73907470703125 -363.73907470703125
318.44000244140625 -363.72857666015625 -363.7393798828125 -363.7385559082031 -363.7397155761719
318.1600036621094 -363.7286071777344 -363.7393493652344 -363.73858642578125 -363.74029541015625
318.7066650390625 -363.7286682128906 -363.7393798828125 -363.7388000488281 -363.73992919921875
318.3666687011719 -363.7286376953125 -363.7393798828125 -363.737548828125 -363.7375793457031
318.3800048828125 -363.7286071777344 -363.7394104003906 -363.7389831542969 -363.7396240234375
316.5533447265625 -363.7286071777344 -363.7393798828125 -363.73846435546875 -363.74169921875
318.02667236328125 -363.7285461425781 -363.7393493652344 -363.74017333984375 -363.73956298828125
318.49334716796875 -363.7286071777344 -363.73937988281

317.7799987792969 -363.7289123535156 -363.7393493652344 -363.7406921386719 -363.7376708984375
317.52001953125 -363.7290344238281 -363.7393493652344 -363.7397766113281 -363.7389221191406
318.2933349609375 -363.72900390625 -363.7393493652344 -363.73992919921875 -363.74127197265625
319.606689453125 -363.7290344238281 -363.7394104003906 -363.74127197265625 -363.7380676269531
319.02667236328125 -363.7289733886719 -363.7393798828125 -363.73944091796875 -363.73919677734375
318.260009765625 -363.7289733886719 -363.7394714355469 -363.7397766113281 -363.7384033203125
317.9800109863281 -363.7289733886719 -363.7393798828125 -363.7411804199219 -363.7391662597656
318.97332763671875 -363.7289733886719 -363.7394104003906 -363.7384948730469 -363.740966796875
319.99334716796875 -363.7290344238281 -363.7393493652344 -363.73956298828125 -363.73876953125
318.90667724609375 -363.7289733886719 -363.7394104003906 -363.7384948730469 -363.7393798828125
320.4200134277344 -363.7289733886719 -363.7394714355469 -36

318.2266845703125 -363.7296142578125 -363.7393798828125 -363.7401123046875 -363.7397766113281
317.6266784667969 -363.7295837402344 -363.7393493652344 -363.7381591796875 -363.7377624511719
319.58001708984375 -363.72967529296875 -363.7394104003906 -363.7418212890625 -363.73822021484375
318.26666259765625 -363.7296142578125 -363.7393493652344 -363.73895263671875 -363.7381591796875
318.0733337402344 -363.7295837402344 -363.7392272949219 -363.73968505859375 -363.7396240234375
316.32666015625 -363.7295837402344 -363.7393798828125 -363.73907470703125 -363.740234375
319.14666748046875 -363.72955322265625 -363.7393493652344 -363.7393493652344 -363.7389221191406
317.52667236328125 -363.7294921875 -363.7393493652344 -363.73809814453125 -363.7366638183594
319.3066711425781 -363.72943115234375 -363.7394104003906 -363.7390441894531 -363.7386474609375
317.9266662597656 -363.72943115234375 -363.7393493652344 -363.7413330078125 -363.738525390625
319.40667724609375 -363.7293701171875 -363.7393798828125 

In [ ]:
train(npoints = 13, batchsize = 1000, nsamples = 8) #thresholding

15.93700122833252 -14.505213737487793 -15.083515167236328 -14.554069519042969 -14.75048542022705 -14.75048542022705
11.77400016784668 -14.417060852050781 -15.070755004882812 -14.386701583862305 -14.704880714416504 -14.704880714416504
11.212000846862793 -12.968181610107422 -14.738553047180176 -12.984586715698242 -15.757709503173828 -15.757710456848145
11.391000747680664 -10.286625862121582 -13.25163745880127 -11.3473482131958 -12.711153030395508 -12.711151123046875
11.261000633239746 -6.608799934387207 -9.754908561706543 -6.212357044219971 -8.69325065612793 -8.69325065612793
11.54800033569336 -7.599280834197998 -10.005451202392578 -7.787478446960449 -10.820524215698242 -10.82052230834961
11.441000938415527 -6.108944416046143 -8.365155220031738 -4.55109167098999 -8.464164733886719 -8.464165687561035
11.26400089263916 -4.702649116516113 -7.2387285232543945 -5.506749153137207 -7.7887773513793945 -7.788776874542236
11.257000923156738 -5.0194220542907715 -7.738556861877441 -6.037553787231445

11.13900089263916 -0.8667780756950378 -1.6071076393127441 -1.3354415893554688 -1.980041742324829 -1.980043649673462
11.094000816345215 -0.904059886932373 -1.6741749048233032 -0.08588790893554688 -0.10838061571121216 -0.10838037729263306
11.106000900268555 -0.9631761908531189 -1.7110440731048584 -1.0762357711791992 -0.5510168075561523 -0.5510168075561523
11.192000389099121 -1.032251000404358 -1.8003698587417603 -0.4627358913421631 -3.207479476928711 -3.2074813842773438
11.220000267028809 -0.973299503326416 -1.714524507522583 -1.0471420288085938 -1.6802184581756592 -1.6802233457565308
11.118000984191895 -0.931056797504425 -1.6870033740997314 -0.8339881896972656 -1.466602087020874 -1.466613531112671
11.154000282287598 -0.8808892369270325 -1.6164274215698242 -0.7358713150024414 -0.6477756500244141 -0.6477756500244141
11.157000541687012 -0.8313913345336914 -1.5333203077316284 -1.3739471435546875 -0.9869899749755859 -0.9869899749755859
11.092000961303711 -0.8023645281791687 -1.49838757514953

10.86400032043457 -1.2693655490875244 -2.2192940711975098 -1.1019413471221924 -0.25783491134643555 -0.25783872604370117
10.782000541687012 -1.1835243701934814 -2.0703704357147217 -1.181929349899292 -5.010982990264893 -5.010975360870361
10.859000205993652 -1.284708023071289 -2.2148308753967285 -0.9685368537902832 -1.3791069984436035 -1.3791069984436035
10.879000663757324 -1.5374094247817993 -2.55476975440979 -0.7051687240600586 -2.0113344192504883 -2.0113344192504883
10.689000129699707 -1.7971783876419067 -2.9510791301727295 -0.9445362091064453 -1.6168798208236694 -1.6168837547302246
10.859000205993652 -2.0260472297668457 -3.274643659591675 -0.9948348999023438 -5.424794673919678 -5.424790382385254
10.707000732421875 -2.1156251430511475 -3.4163384437561035 -2.7155075073242188 -1.4621105194091797 -1.4621143341064453
10.827000617980957 -1.9746670722961426 -3.1900758743286133 -1.8952388763427734 -2.75396728515625 -2.7539596557617188
10.797000885009766 -1.6053489446640015 -2.6828198432922363

10.76400089263916 -0.7711475491523743 -1.4863094091415405 -0.7763996124267578 -0.6608295440673828 -0.6608257293701172
10.759000778198242 -0.8649226427078247 -1.6305030584335327 -1.6679763793945312 -0.37288856506347656 -0.37288856506347656
10.931000709533691 -0.9738336801528931 -1.8152323961257935 -0.29096031188964844 -0.31386756896972656 -0.31386756896972656
10.699000358581543 -1.100358247756958 -2.0439023971557617 -1.693882942199707 -3.099604606628418 -3.099606513977051
10.558000564575195 -1.2025889158248901 -2.2434158325195312 -0.9278224110603333 -2.3185348510742188 -2.3185272216796875
10.641000747680664 -1.275352954864502 -2.3024373054504395 -0.0775146484375 -2.556741714477539 -2.5567493438720703
10.532000541687012 -1.3187751770019531 -2.3643364906311035 -1.7605247497558594 -4.090232849121094 -4.090244293212891
10.666000366210938 -1.2645494937896729 -2.282355785369873 -1.8860664367675781 -0.6059017181396484 -0.6059017181396484
10.567000389099121 -1.1646965742111206 -2.09331226348876

10.76300048828125 -0.8737778663635254 -1.6227667331695557 -1.311605453491211 -1.5942344665527344 -1.5942306518554688
10.767000198364258 -0.8880627155303955 -1.675713062286377 -0.8144454956054688 -1.927520751953125 -1.9275245666503906
10.773000717163086 -0.9464552998542786 -1.7540638446807861 -0.6082305908203125 -0.6977119445800781 -0.6977119445800781
10.579000473022461 -1.0066756010055542 -1.8515989780426025 -1.4454097747802734 -0.410186767578125 -0.4101829528808594
10.654000282287598 -1.070181965827942 -1.9156057834625244 -1.6333427429199219 -0.8930721282958984 -0.8930683135986328
10.749000549316406 -1.1182359457015991 -1.9779284000396729 -1.7087936401367188 -1.4396247863769531 -1.4396286010742188
10.708000183105469 -1.1208348274230957 -1.9969269037246704 -0.3400535583496094 -1.8669700622558594 -1.8669624328613281
10.72800064086914 -1.103232502937317 -1.9430291652679443 -0.4096355438232422 -2.494915008544922 -2.4949073791503906
10.672000885009766 -1.1623725891113281 -2.056652784347534

...

11.042000770568848 -0.4993264377117157 -0.9337970018386841 -0.7359619140625 -0.4697723388671875 -0.469757080078125
11.04800033569336 -0.48027899861335754 -0.9168314337730408 -0.005340576171875 -5.224822998046875 -5.224945068359375
11.052000999450684 -0.4899972081184387 -0.8899441361427307 -1.477569580078125 -0.42498779296875 -0.42498779296875
11.157000541687012 -0.5230936408042908 -0.9971421957015991 -0.434539794921875 -0.0015869140625 -0.0015869140625
11.13800048828125 -0.5363588333129883 -1.0212152004241943 -0.035186767578125 -1.485931396484375 -1.485931396484375
11.12600040435791 -0.5675053596496582 -1.0465549230575562 -0.26507568359375 -1.7637786865234375 -1.7637481689453125
11.023000717163086 -0.5751511454582214 -1.072555422782898 -0.176666259765625 -1.5559844970703125 -1.5559539794921875
11.107000350952148 -0.651240885257721 -1.1729687452316284 -1.486480712890625 -2.17706298828125 -2.177001953125
11.18600082397461 -0.7001389265060425 -1.2650030851364136 -1.03985595703125 -0.48312

10.93600082397461 -0.6894127726554871 -1.2336335182189941 -0.0196533203125 -0.121246337890625 -0.12127685546875
11.218000411987305 -0.6634501218795776 -1.2329936027526855 -0.848876953125 -0.079803466796875 -0.079803466796875
11.158000946044922 -0.6206445097923279 -1.1430402994155884 -0.418487548828125 -1.8602294921875 -1.86029052734375
11.06100082397461 -0.69423508644104 -1.272655963897705 -0.83648681640625 -1.425048828125 -1.42498779296875
11.225000381469727 -0.6676985025405884 -1.2774683237075806 -1.2548828125 -2.59234619140625 -2.59234619140625
11.175000190734863 -0.6980485916137695 -1.3167940378189087 -2.36871337890625 -2.452484130859375 -2.452545166015625
11.010000228881836 -0.7420821785926819 -1.3621890544891357 -0.545745849609375 -0.28607177734375 -0.28607177734375
10.983000755310059 -0.7235559225082397 -1.3201395273208618 -0.345977783203125 -0.1185302734375 -0.1185302734375
10.981000900268555 -0.7079963684082031 -1.3118658065795898 -1.03045654296875 -0.8011474609375 -0.80114746

11.040000915527344 -0.6757852435112 -1.2294613122940063 -1.013641357421875 -0.66058349609375 -0.660675048828125
11.042000770568848 -0.6698594093322754 -1.2059898376464844 -0.258575439453125 -0.24444580078125 -0.24444580078125
11.009000778198242 -0.6397429704666138 -1.1662626266479492 -0.91876220703125 -2.773651123046875 -2.773651123046875
11.042000770568848 -0.6425215005874634 -1.188982367515564 -0.3956298828125 -0.419586181640625 -0.419586181640625
11.0260009765625 -0.629172146320343 -1.1426053047180176 -0.10968017578125 -2.44244384765625 -2.442413330078125
11.17400074005127 -0.623652458190918 -1.1469005346298218 -0.17041015625 -0.197479248046875 -0.197509765625
10.987000465393066 -0.6515746116638184 -1.1874984502792358 -1.055694580078125 -2.19415283203125 -2.19415283203125
11.193000793457031 -0.644385576248169 -1.1731356382369995 -0.039581298828125 -0.24755859375 -0.247589111328125
11.134000778198242 -0.6718218326568604 -1.1918247938156128 -0.191619873046875 -0.416778564453125 -0.416

11.011000633239746 -0.6944176554679871 -1.2386409044265747 -0.343048095703125 -0.1427001953125 -0.1427001953125
11.079000473022461 -0.7299376726150513 -1.344200611114502 -0.53045654296875 -0.649139404296875 -0.649139404296875
11.093000411987305 -0.7567571997642517 -1.3385480642318726 -0.4412841796875 -3.43402099609375 -3.433990478515625
11.172000885009766 -0.798902153968811 -1.4205780029296875 -0.17938232421875 -0.548828125 -0.548828125
11.111000061035156 -0.8346013426780701 -1.4905565977096558 -0.113861083984375 -4.102874755859375 -4.1029052734375
11.009000778198242 -0.8161660432815552 -1.4880362749099731 -0.81756591796875 -0.820648193359375 -0.820648193359375
10.967000961303711 -0.8021219372749329 -1.4274365901947021 -0.4791259765625 -1.0279541015625 -1.0279541015625
11.041000366210938 -0.7898207306861877 -1.4595293998718262 -0.582244873046875 -0.4378662109375 -0.437896728515625
11.04800033569336 -0.7510527968406677 -1.3733798265457153 -0.64227294921875 -0.89794921875 -0.89794921875


11.120000839233398 -0.7776873707771301 -1.433197021484375 -1.7158203125 -0.145111083984375 -0.145111083984375
11.125000953674316 -0.8340216279029846 -1.5687851905822754 -1.881256103515625 -4.279998779296875 -4.2799072265625
11.134000778198242 -0.7732129693031311 -1.4689037799835205 -0.596923828125 -0.93463134765625 -0.93463134765625
10.969000816345215 -0.8116304278373718 -1.5262174606323242 -1.6417236328125 -1.30682373046875 -1.306884765625
11.067000389099121 -0.8109763860702515 -1.5147762298583984 -0.010711669921875 -2.619232177734375 -2.6192626953125
11.130000114440918 -0.8238617181777954 -1.5665357112884521 -0.14764404296875 -0.087646484375 -0.087646484375
10.873000144958496 -0.8297122120857239 -1.5339374542236328 -1.223388671875 -2.254058837890625 -2.254058837890625
10.943000793457031 -0.7942862510681152 -1.5101209878921509 -0.890594482421875 -1.451507568359375 -1.451507568359375
11.051000595092773 -0.7333146333694458 -1.404807448387146 -0.793548583984375 -0.426025390625 -0.4260253

In [ ]:
train(npoints = 13, batchsize = 1000, nsamples = 8) #thresholding binary

13.910000801086426 -14.906752586364746 -15.102460861206055 -14.99688720703125 -15.03282642364502 -15.03282642364502
11.480000495910645 -14.124567985534668 -15.041982650756836 -14.311320304870605 -14.400656700134277 -14.400655746459961
12.652000427246094 -13.042152404785156 -14.786443710327148 -12.90709114074707 -16.219497680664062 -16.219497680664062
11.625000953674316 -10.124996185302734 -13.183647155761719 -9.049715995788574 -13.261316299438477 -13.261314392089844
11.290000915527344 -6.5390238761901855 -9.6568021774292 -6.133817672729492 -8.238977432250977 -8.23897933959961
11.981000900268555 -9.780517578125 -12.20201587677002 -10.957145690917969 -10.051321983337402 -10.051321029663086
11.4660005569458 -6.7268147468566895 -9.160473823547363 -7.928413391113281 -9.49146556854248 -9.491463661193848
11.419000625610352 -4.73626184463501 -7.0627899169921875 -5.842904567718506 -6.848125457763672 -6.848124027252197
11.345000267028809 -4.442760467529297 -6.893574237823486 -3.0416312217712402 

11.187000274658203 -1.5484496355056763 -2.5955934524536133 -1.9612464904785156 -0.9834303855895996 -0.9834303855895996
11.157000541687012 -1.5552823543548584 -2.5929360389709473 -1.7922916412353516 -1.99125337600708 -1.9912590980529785
11.20300006866455 -1.3035160303115845 -2.264036178588867 -1.5388500690460205 -3.981497287750244 -3.9815030097961426
10.984000205993652 -1.1883721351623535 -2.133734703063965 -0.3518238067626953 -0.6857759952545166 -0.6857762336730957
11.076000213623047 -1.1040425300598145 -2.047077178955078 -0.9798521995544434 -2.2814016342163086 -2.28140926361084
11.145000457763672 -1.1612881422042847 -2.110750436782837 -0.2581758499145508 -0.7834429740905762 -0.7834467887878418
11.202000617980957 -1.1802396774291992 -2.192047595977783 -2.1443352699279785 -1.8869743347167969 -1.8869781494140625
11.272000312805176 -1.2203749418258667 -2.252699136734009 -0.14928936958312988 -0.8361063003540039 -0.8361063003540039
11.099000930786133 -1.3037469387054443 -2.3909189701080322 

10.813000679016113 -2.2245218753814697 -3.6123626232147217 -3.7061214447021484 -3.4141464233398438 -3.4141464233398438
10.773000717163086 -2.2687747478485107 -3.715912103652954 -1.5358753204345703 -4.873786926269531 -4.873785018920898
10.832000732421875 -2.549132823944092 -4.139719009399414 -4.585300445556641 -2.7215728759765625 -2.721571922302246
10.869000434875488 -2.595339059829712 -4.178849697113037 -2.847930908203125 -4.873283386230469 -4.87327766418457
10.819000244140625 -2.600767135620117 -4.275258541107178 -1.7271976470947266 -4.529850006103516 -4.529844284057617
10.78700065612793 -2.683974504470825 -4.436880588531494 -3.2080259323120117 -4.811941146850586 -4.811941146850586
10.734000205993652 -2.660813570022583 -4.441982746124268 -4.883089065551758 -8.229965209960938 -8.229961395263672
10.564000129699707 -2.3899033069610596 -4.118960380554199 -2.2734851837158203 -6.230979919433594 -6.230974197387695
10.544000625610352 -2.2258267402648926 -3.877652406692505 -1.0367565155029297 

10.714000701904297 -1.4787166118621826 -2.6512398719787598 -1.5121560096740723 -4.133546829223633 -4.133543014526367
10.753000259399414 -1.4648419618606567 -2.652003526687622 -1.5882606506347656 -2.675283432006836 -2.6752872467041016
10.679000854492188 -1.5145670175552368 -2.6990792751312256 -1.1836624145507812 -3.286933660507202 -3.2869298458099365
10.733000755310059 -1.5698987245559692 -2.804356813430786 -0.7918910980224609 -5.846029281616211 -5.846035003662109
10.742000579833984 -1.7108325958251953 -3.062765121459961 -0.7784557342529297 -1.1347479820251465 -1.1347441673278809
10.724000930786133 -1.8672752380371094 -3.284194231033325 -1.6935386657714844 -1.763284683227539 -1.763284683227539
10.679000854492188 -1.8558577299118042 -3.2755839824676514 -0.7729787826538086 -4.630339622497559 -4.630344390869141
10.768000602722168 -1.9126713275909424 -3.331944227218628 -2.0044631958007812 -2.830411911010742 -2.8304100036621094
10.842000961303711 -1.9104948043823242 -3.3003854751586914 -1.59

10.709000587463379 -1.8393583297729492 -3.1693170070648193 -1.797640323638916 -2.1631531715393066 -2.1631569862365723
10.63900089263916 -1.9822776317596436 -3.3851072788238525 -0.5438833236694336 -1.2416973114013672 -1.2416973114013672
10.758000373840332 -2.0446836948394775 -3.4990718364715576 -1.9995384216308594 -4.410671234130859 -4.410663604736328
10.817000389099121 -2.0945627689361572 -3.542057752609253 -1.6591110229492188 -1.536520004272461 -1.5365276336669922
10.971000671386719 -1.8562374114990234 -3.2139458656311035 -0.8775501251220703 -3.811634063720703 -3.8116455078125
10.973000526428223 -1.7405394315719604 -3.0280487537384033 -2.330108642578125 -2.7962722778320312 -2.7962722778320312
10.981000900268555 -1.8891340494155884 -3.201026439666748 -0.9660568237304688 -5.843820571899414 -5.843820571899414
10.81100082397461 -2.14768123626709 -3.4984285831451416 -2.4128684997558594 -2.5239810943603516 -2.5239810943603516
10.75200080871582 -2.316079616546631 -3.7833995819091797 -1.39186

...

7.360000133514404 -1.108726143836975 -2.190565586090088 -0.8952702283859253 -3.1011571884155273 -3.1011579036712646
7.3560004234313965 -1.0787185430526733 -2.1807644367218018 -1.1106336116790771 -5.246089935302734 -5.246090888977051
7.38200044631958 -1.0875576734542847 -2.1843740940093994 -0.26054322719573975 -7.400974273681641 -7.40097713470459
7.342000484466553 -1.1013871431350708 -2.164419412612915 -1.6579246520996094 -7.80112886428833 -7.801127910614014
7.391000270843506 -1.089809775352478 -2.1940722465515137 -0.4020223617553711 -3.399106979370117 -3.3991074562072754
7.332000255584717 -1.0816731452941895 -2.164982318878174 -0.952601969242096 -1.0442609786987305 -1.0442607402801514
7.453000545501709 -1.0877043008804321 -2.168902635574341 -1.0558756589889526 -0.3258240222930908 -0.32582521438598633
7.381000518798828 -1.085527777671814 -2.1694414615631104 -1.411318302154541 -2.6556129455566406 -2.6556143760681152
7.3580002784729 -1.090000867843628 -2.1735475063323975 -0.46737557649612

7.3520002365112305 -1.1253368854522705 -2.2535784244537354 -1.2306666374206543 -0.6307152509689331 -0.6307152509689331
7.340000152587891 -1.1215275526046753 -2.2224292755126953 -0.9872123003005981 -6.2487382888793945 -6.248739242553711
7.272000312805176 -1.1116975545883179 -2.1929328441619873 -0.41525912284851074 -2.616774320602417 -2.616774559020996
7.353000164031982 -1.0788979530334473 -2.17128324508667 -0.8176121711730957 -2.4002838134765625 -2.4002835750579834
7.378000259399414 -1.0916950702667236 -2.148444652557373 -0.15126347541809082 -1.74454665184021 -1.744549036026001
7.306000232696533 -1.0876538753509521 -2.159698724746704 -1.8959439992904663 -0.8682007789611816 -0.8682003021240234
7.368000507354736 -1.0513839721679688 -2.1640827655792236 -0.9226248264312744 -1.5187866687774658 -1.5187852382659912
7.315000534057617 -1.099259376525879 -2.209866762161255 -1.612791895866394 -3.0081958770751953 -3.008194923400879
7.303000450134277 -1.0913463830947876 -2.198450803756714 -0.2025992

7.311000347137451 -1.1915035247802734 -2.2569494247436523 -0.7993865013122559 -1.7446932792663574 -1.744692087173462
7.448000431060791 -1.1505720615386963 -2.315471887588501 -1.7186498641967773 -1.3508739471435547 -1.3508758544921875
7.372000217437744 -1.1229273080825806 -2.2098817825317383 -1.1103863716125488 -0.7062265276908875 -0.7062249779701233
7.340000152587891 -1.1170579195022583 -2.2201809883117676 -0.0705873966217041 -2.9143640995025635 -2.9143660068511963
7.385000228881836 -1.0894136428833008 -2.153796672821045 -0.3710819482803345 -0.35517752170562744 -0.35517752170562744
7.4120001792907715 -1.0850766897201538 -2.164663314819336 -1.9027867317199707 -0.3083707094192505 -0.3083704710006714
7.362000465393066 -1.102807879447937 -2.1523640155792236 -1.190739393234253 -3.009488582611084 -3.009488105773926
7.409000396728516 -1.0401811599731445 -2.112734317779541 -0.7285008430480957 -1.990318775177002 -1.990318775177002
7.439000129699707 -1.0650821924209595 -2.127011775970459 -1.7214

7.378000259399414 -1.0900027751922607 -2.167365074157715 -0.33387160301208496 -2.5525059700012207 -2.5525097846984863
7.335000514984131 -1.0380290746688843 -2.1157984733581543 -0.6954932808876038 -0.13501089811325073 -0.13501089811325073
7.387000560760498 -1.056646466255188 -2.10304594039917 -1.0646588802337646 -4.0806355476379395 -4.080634593963623
7.333000183105469 -1.0808030366897583 -2.1506640911102295 -1.263748049736023 -6.527456760406494 -6.527457237243652
7.378000259399414 -1.1237410306930542 -2.2102668285369873 -0.5563195943832397 -0.33655834197998047 -0.33655834197998047
7.414000511169434 -1.1154316663742065 -2.2155845165252686 -2.796867609024048 -1.95975661277771 -1.9597537517547607
7.403000354766846 -1.101285696029663 -2.233494758605957 -0.7961111068725586 -1.952362060546875 -1.9523630142211914
7.419000148773193 -1.1776683330535889 -2.350191593170166 -1.2341508865356445 -0.37892651557922363 -0.37892603874206543
7.345000267028809 -1.1251931190490723 -2.2897236347198486 -1.914

7.303000450134277 -1.1513400077819824 -2.3287739753723145 -1.4881149530410767 -0.6155195236206055 -0.6155200004577637
7.340000152587891 -1.1865848302841187 -2.376234531402588 -2.2921643257141113 -0.57753986120224 -0.5775397419929504
7.301000118255615 -1.1366246938705444 -2.2614269256591797 -0.5027259588241577 -0.8755699396133423 -0.8755698204040527
